<a href="https://colab.research.google.com/github/vtecftwy/utseus-dives/blob/main/nbs/dive_2_1_NER_token_classification_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract names entities from documents

In this notebook we will use a pretrained large language model and fine tune is for the specific task of Named Entity Recognition (NER). NER is one type of Token Classification problem in NLP, i.e. assigning a label to each token in a document.

In [1]:
# Transformers installation
try:
    from datasets import __version__
    print(__version__)
except ModuleNotFoundError:
    print('need to install transformers and related packages')
    ! pip install -qq transformers datasets accelerate huggingface_hub evaluate seqeval
    print('all packaged installed')

need to install transformers and related packages
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 27.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that 

# Token classification

Token classification assigns a label to individual tokens in a sentence.

One of the most common token classification tasks is **Named Entity Recognition (NER)**. NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This notebook will:
1. Load a pretrained LLM: [DistilBERT](https://huggingface.co/distilbert-base-uncased), which is a BERT encoder only model processed to be a little smaller without loosing much performance. Encoder only LLMs have access to the left and right side of a token. Therefore, they are good for many problems that do not required text generation. Classification is one of these.

2. Finetune DistilBERT on a specfic dataset, [WNUT 17](https://huggingface.co/datasets/wnut_17), in order to allow it to detect new named entities.
3. Use your finetuned model for inference and validation



<details>
<summary>Technical note</summary>
<p>Token classification is supported by many other model architectures. For instance:</p>
<ul>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/albert">ALBERT</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/bert"> BERT</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/biogpt"> BioGpt</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/bloom"> BLOOM</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/camembert">CamemBERT</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/deberta">DeBERTa</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/distilbert">DistilBERT</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/ernie">ERNIE</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/gpt2"> OpenAI GPT-2</a></li>
<li><a href="https://huggingface.co/docs/transformers/main/model_doc/roberta"> RoBERTa</a></li>
</ul>
</details>




If you have a huggingface account, enter your token to login when prompted. If you do nto have an account, you still can access this distilBert model.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load WNUT 17 dataset

Start by loading the WNUT 17 dataset from the 🤗 Datasets library:

In [3]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for wnut_17 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wnut_17.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

Then take a look at first sample in the dataset:


In [5]:
wnut["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)}

There are thre features for each datapoint: `id`, `tokens` and `ner_tags`

In [13]:
print(' | '.join(wnut['train']['tokens'][0]))

@paulwalk | It | 's | the | view | from | where | I | 'm | living | for | two | weeks | . | Empire | State | Building | = | ESB | . | Pretty | bad | storm | here | last | evening | .


In [12]:
print(wnut['train']['ner_tags'][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]


Each number in `ner_tags` represents an entity. Convert the numbers to their label names to find out what the entities are:

In [14]:
label_list = wnut["train"].features['ner_tags'].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

The letter that prefixes each `ner_tag` indicates the token position of the entity:

- `B-` indicates the beginning of an entity.
- `I-` indicates a token is contained inside the same entity (for example, the `State` token is a part of an entity like
  `Empire State Building`).
- `0` indicates the token doesn't correspond to any entity.

## Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the `tokens` field:

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet, it is just split into words. But it still needs to be further tokenized in subsords. This can be done by setting `is_split_into_words=True` to tokenize the words into subwords. For example:

In [23]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from']

In [24]:
print(' | '.join(wnut['train']['tokens'][0]))

print(' | '.join(tokens))

@paulwalk | It | 's | the | view | from | where | I | 'm | living | for | two | weeks | . | Empire | State | Building | = | ESB | . | Pretty | bad | storm | here | last | evening | .
[CLS] | @ | paul | ##walk | it | ' | s | the | view | from | where | i | ' | m | living | for | two | weeks | . | empire | state | building | = | es | ##b | . | pretty | bad | storm | here | last | evening | . | [SEP]


However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [25]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [26]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length. Indeed, if a few documents are very long and the others are small, only those batches with long documents will use a large memory.

In [27]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [28]:
import evaluate

seqeval = evaluate.load("seqeval")

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [29]:
import numpy as np

labels = [label_list[i] for i in example["ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [30]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [31]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=13,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [32]:
nb_epochs = 20
lr = 1e-5

training_args = TrainingArguments(
    output_dir="my_experiment_wnut_model",
    learning_rate=lr,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=nb_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.301287,0.450617,0.135310,0.208125,0.933821
2,No log,0.270610,0.566553,0.307692,0.398799,0.941730
3,0.244700,0.276679,0.562404,0.338276,0.422454,0.943953
4,0.244700,0.261874,0.541507,0.392956,0.455424,0.945834
5,0.076200,0.290469,0.572581,0.394810,0.467361,0.947629
6,0.076200,0.286785,0.552984,0.420760,0.477895,0.947971
7,0.076200,0.294221,0.529894,0.418906,0.467909,0.946903
8,0.041700,0.307659,0.538182,0.411492,0.466387,0.946646
9,0.041700,0.323708,0.569921,0.400371,0.470332,0.947330
10,0.025300,0.334282,0.544767,0.400371,0.461538,0.946774


TrainOutput(global_step=4260, training_loss=0.0512973364809869, metrics={'train_runtime': 553.3991, 'train_samples_per_second': 122.66, 'train_steps_per_second': 7.698, 'total_flos': 918778110039120.0, 'train_loss': 0.0512973364809869, 'epoch': 20.0})

For a more in-depth example of how to finetune a model for token classification, take a look at the Huggingface
[notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb).

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [33]:
from transformers import pipeline
finetuned_model_name = 'your_account/my_experiment_wnut_model'

# classifier = pipeline("ner", model=finetuned_model_name)
classifier = pipeline("ner", model=trainer.model.to('cpu'), tokenizer=trainer.tokenizer)
# res = classifier(text);

In [34]:
texts = {
    1: "The Golden State Warriors are an American professional basketball team based in San Francisco.",
    2: "Shanghai is a main city in China. The Pearl Tower is a famous landmark in the city. Shanghai Pudong Development Bank is a main financial institution here.",
    3: "One of the most famous companies located in Cincinatti is Procter and Gamble",
    4: "It's the view from where I'm living for two weeks.  Empire State Building = ESB.  Pretty bad storm here last evening."
}

In [35]:
for text in texts.values():
    print(text)
    res = classifier(text)
    print('\n'.join([f" {v['word']}  ({v['entity']})" for v in res ]))
    print()

The Golden State Warriors are an American professional basketball team based in San Francisco.
 the  (B-group)
 golden  (B-location)
 state  (I-location)
 warriors  (I-group)
 san  (B-location)
 francisco  (I-location)

Shanghai is a main city in China. The Pearl Tower is a famous landmark in the city. Shanghai Pudong Development Bank is a main financial institution here.
 shanghai  (B-location)
 china  (B-location)
 shanghai  (B-location)
 pu  (B-location)
 ##dong  (I-location)

One of the most famous companies located in Cincinatti is Procter and Gamble
 ci  (B-location)
 ##tti  (I-location)
 pro  (B-corporation)
 ##cter  (B-corporation)
 and  (I-corporation)
 gamble  (I-corporation)

It's the view from where I'm living for two weeks.  Empire State Building = ESB.  Pretty bad storm here last evening.
 empire  (B-location)
 state  (I-location)
 building  (I-location)
 es  (B-location)



In [36]:
class render_tokens:
    def __init__(self, res):
        self.res_original = res
        self.len = len(res)
        self.res = res.copy()

    def __call__(self):
        token = self.res[0]['word']
        label = self.res[0]['entity']


You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [37]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")
tokenizer = trainer.tokenizer
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [38]:
import torch
from transformers import AutoModelForTokenClassification

# model = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")
# with torch.no_grad():
#     logits = model(**inputs).logits

model = trainer.model
with torch.no_grad():
    outputs = model(**inputs)

outputs.keys()

odict_keys(['logits'])

In [39]:
logits = outputs.logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [40]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']